# Precess Raw Knowledge Graphs in Clickhouse

In [1]:
from clickhouse_driver import Client
from pandas import DataFrame

In [2]:
CLICKHOUSE_HOST = 'localhost'
COMPRESSION = 'lz4'
DROP_TABLE = True
CREATE_TABLE = True
INSERT_INTO_TABLES = True
client = Client(host=CLICKHOUSE_HOST)
execute = client.execute
DIR_URL = 'CaLiGraph/'

## Drop and Create the knowledge_graph_raw table

In [3]:
def load_from_ch_to_df(sql_query, columns=None):
    return DataFrame(execute(sql_query), columns=columns)

if DROP_TABLE:
    execute('DROP TABLE IF EXISTS knowledge_graph_processed')
    execute('DROP TABLE IF EXISTS subject_processed_agg')
    execute('DROP TABLE IF EXISTS filtered_processed_nodes')
    execute('DROP TABLE IF EXISTS knowledge_graph_processed_filtered')

if CREATE_TABLE:
    execute('''
            CREATE TABLE IF NOT EXISTS knowledge_graph_processed(
                source String,
                subject String,
                predicate String,
                object String)
            ENGINE = ReplacingMergeTree()
            ORDER BY (source, subject, predicate, object);
            ''')
    execute('''
            CREATE TABLE IF NOT EXISTS subject_processed_agg(
                subject String,
                number Int64)
            ENGINE = ReplacingMergeTree()
            ORDER BY (subject);
            ''')
    execute('''
            CREATE TABLE IF NOT EXISTS filtered_processed_nodes(
                node String)
            ENGINE = ReplacingMergeTree()
            ORDER BY (node);
            ''')
    execute('''
            CREATE TABLE IF NOT EXISTS knowledge_graph_processed_filtered(
                source String,
                subject String,
                predicate String,
                object String)
            ENGINE = ReplacingMergeTree()
            ORDER BY (source, subject, predicate, object);
            ''')

## Process Knowledge Graph

In [4]:
if INSERT_INTO_TABLES:
    execute('''
            INSERT INTO knowledge_graph_processed
            SELECT
                source,
                startsWith(subject, '<')? splitByChar('=', splitByChar('#', splitByChar('/', replace(subject, '>', ''))[-1])[-1])[-1]: replace(subject, '"', '') AS subject,
                startsWith(predicate, '<')? splitByChar('=', splitByChar('#', splitByChar('/', replace(predicate, '>', ''))[-1])[-1])[-1]: replace(predicate, '"', '') AS predicate,
                startsWith(object, '<')? splitByChar('=', splitByChar('#', splitByChar('/', replace(object, '>', ''))[-1])[-1])[-1]: replace(object, '"', '') AS object
            FROM knowledge_graph_raw;
            ''')
    execute('''
            INSERT INTO subject_processed_agg
            SELECT
                subject,
                count() AS number
            FROM knowledge_graph_processed
            GROUP BY subject;
            ''')
    execute('''
            INSERT INTO filtered_processed_nodes
            SELECT t1.node AS node
            FROM (
                SELECT
                    subject AS node
                FROM subject_processed_agg) t1
            ALL INNER JOIN (
                SELECT
                    DISTINCT object AS node
                FROM knowledge_graph_processed
                PREWHERE subject != object) t2
            USING node;
            ''')
    execute('''
            INSERT INTO knowledge_graph_processed_filtered
            SELECT *
            FROM knowledge_graph_processed
            PREWHERE subject IN (SELECT node FROM filtered_processed_nodes)
                 AND object IN (SELECT node FROM filtered_processed_nodes)
                 AND subject != object;
            ''')

## Base Graph Parameters

In [28]:
def base_graph_parameters(knowledge_graph_table_name):
    return load_from_ch_to_df('''
                SELECT
                    count(),
                    'Links'
                FROM {0}
                UNION ALL
                SELECT
                    count(DISTINCT subject),
                    'Subjects'
                FROM {0}
                UNION ALL
                SELECT
                    count(DISTINCT predicate),
                    'Predicates'
                FROM {0}
                UNION ALL
                SELECT
                    count(DISTINCT object),
                    'Objects'
                FROM {0};
               '''.format(knowledge_graph_table_name),
               columns=['Number of unique items', 'Item types'])

def number_of_links_by_type_items(item_type, knowledge_graph_table_name, limit=10):
    return load_from_ch_to_df('''
                SELECT
                    {0},
                    count() AS number_of_links
                FROM {1}
                GROUP BY {0}
                ORDER BY number_of_links DESC
                LIMIT {2};
                '''.format(item_type, knowledge_graph_table_name, limit),
                columns=[item_type, 'Number of Links'])

def print_graph_parameters(knowledge_graph_table_name):
    print('\nBase Graph Parameters\n', base_graph_parameters(knowledge_graph_table_name).to_markdown())
    for item_type in ['subject', 'predicate', 'object']:
        print(f'\nTop {item_type} in the Graph\n', number_of_links_by_type_items(item_type, knowledge_graph_table_name).to_markdown())

### Raw Graph

In [30]:
print_graph_parameters('knowledge_graph_raw')


Base Graph Parameters
 |    |   Number of unique items | Item types   |
|---:|-------------------------:|:-------------|
|  0 |                 12072467 | Subjects     |
|  1 |                      554 | Predicates   |
|  2 |                125638829 | Links        |
|  3 |                 15728434 | Objects      |

Top subject in the Graph
 |    | subject                                                             |   Number of Links |
|---:|:--------------------------------------------------------------------|------------------:|
|  0 | <http://caligraph.org/ontology/Person>                              |              1151 |
|  1 | <http://caligraph.org/ontology/Person_associated_with_places>       |               615 |
|  2 | <http://caligraph.org/resource/Peregrine_falcon>                    |               610 |
|  3 | <http://caligraph.org/resource/Osprey>                              |               597 |
|  4 | <http://caligraph.org/resource/Barn_swallow>                      

### Processed Graph

In [31]:
print_graph_parameters('knowledge_graph_processed')


Base Graph Parameters
 |    |   Number of unique items | Item types   |
|---:|-------------------------:|:-------------|
|  0 |                  8175366 | Subjects     |
|  1 |                      283 | Predicates   |
|  2 |                125617154 | Links        |
|  3 |                 15079945 | Objects      |

Top subject in the Graph
 |    | subject                       |   Number of Links |
|---:|:------------------------------|------------------:|
|  0 | Person                        |              1163 |
|  1 | Peregrine_falcon              |               619 |
|  2 | Person_associated_with_places |               615 |
|  3 | Senior_High_School            |               615 |
|  4 | Barn_swallow                  |               601 |
|  5 | Osprey                        |               598 |
|  6 | Forced_Labour_Convention      |               593 |
|  7 | Ruddy_turnstone               |               577 |
|  8 | Cattle_egret                  |               575 |
|  9 |

### Processed Filtered Graph

In [32]:
print_graph_parameters('knowledge_graph_processed_filtered')


Base Graph Parameters
 |    |   Number of unique items | Item types   |
|---:|-------------------------:|:-------------|
|  0 |                   861781 | Subjects     |
|  1 |                      206 | Predicates   |
|  2 |                  3284474 | Links        |
|  3 |                   360328 | Objects      |

Top subject in the Graph
 |    | subject                   |   Number of Links |
|---:|:--------------------------|------------------:|
|  0 | S                         |               328 |
|  1 | Osprey                    |               314 |
|  2 | Madonna_(entertainer)     |               290 |
|  3 | 2                         |               288 |
|  4 | Winston_Churchill         |               273 |
|  5 | World_Health_Organization |               270 |
|  6 | Mallard                   |               265 |
|  7 | Ruff                      |               260 |
|  8 | World_Heritage_Site       |               252 |
|  9 | Michael_Jackson           |               2